In [1]:
# Cell 1: Setup
"""
Model Evaluation - Sales Forecasting Performance
Analyzing Prophet model predictions vs actuals
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pickle

print(" Imports complete")

 Imports complete


In [2]:
# Cell 2: Load Results
# Load evaluation data
eval_df = pd.read_csv('../data/output/model_evaluation.csv')
eval_df['ds'] = pd.to_datetime(eval_df['ds'])

# Load full forecast
forecast_df = pd.read_csv('../data/output/forecast_30days.csv')
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])

print(f"Evaluation period: {len(eval_df)} days")
print(f"Forecast period: {len(forecast_df)} days")

Evaluation period: 26 days
Forecast period: 309 days


In [3]:
# Cell 3: Performance Metrics
mape = (abs(eval_df['y'] - eval_df['yhat']) / eval_df['y']).mean() * 100
rmse = np.sqrt(((eval_df['y'] - eval_df['yhat']) ** 2).mean())
mae = abs(eval_df['y'] - eval_df['yhat']).mean()

print("=== MODEL PERFORMANCE ===")
print(f"MAPE: {mape:.2f}%")
print(f"RMSE: ${rmse:,.2f}")
print(f"MAE: ${mae:,.2f}")

# Interpretation
if mape < 10:
    quality = "Excellent"
elif mape < 20:
    quality = "Good"
elif mape < 30:
    quality = "Acceptable"
else:
    quality = "Needs Improvement"

print(f"\nModel Quality: {quality}")

=== MODEL PERFORMANCE ===
MAPE: 48.91%
RMSE: $31,950.16
MAE: $21,457.46

Model Quality: Needs Improvement


In [4]:
# Cell 4: Actual vs Predicted
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=eval_df['ds'], 
    y=eval_df['y'],
    mode='lines+markers',
    name='Actual Sales',
    line=dict(color='blue')
))

# Predicted values
fig.add_trace(go.Scatter(
    x=eval_df['ds'], 
    y=eval_df['yhat'],
    mode='lines',
    name='Predicted Sales',
    line=dict(color='red', dash='dash')
))

# Confidence interval
fig.add_trace(go.Scatter(
    x=eval_df['ds'],
    y=eval_df['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=eval_df['ds'],
    y=eval_df['yhat_lower'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Confidence Interval'
))

fig.update_layout(
    title='Actual vs Predicted Sales (Test Period)',
    xaxis_title='Date',
    yaxis_title='Sales ($)',
    height=500
)

fig.show()

print("\n INSIGHT: How well does the model track actual sales?")


 INSIGHT: How well does the model track actual sales?


In [7]:
# Cell 5: Residual Analysis
eval_df['residual'] = eval_df['y'] - eval_df['yhat']
eval_df['residual_pct'] = (eval_df['residual'] / eval_df['y']) * 100

fig = px.scatter(eval_df, x='yhat', y='residual',
                 title='Residual Plot',
                 labels={'yhat': 'Predicted Sales', 'residual': 'Residual (Actual - Predicted)'})
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.show()

print("\n INSIGHT: Residuals should be randomly distributed around zero")
print(f"Mean residual: ${eval_df['residual'].mean():.2f} (should be close to 0)")


 INSIGHT: Residuals should be randomly distributed around zero
Mean residual: $-8429.41 (should be close to 0)


In [8]:
# Cell 6: Error Distribution
fig = px.histogram(eval_df, x='residual_pct', nbins=20,
                   title='Distribution of Prediction Errors (%)',
                   labels={'residual_pct': 'Percentage Error'})
fig.show()

print(f"\n% of predictions within ±10%: {(abs(eval_df['residual_pct']) <= 10).mean() * 100:.1f}%")
print(f"% of predictions within ±20%: {(abs(eval_df['residual_pct']) <= 20).mean() * 100:.1f}%")


% of predictions within ±10%: 7.7%
% of predictions within ±20%: 38.5%


In [10]:
# Cell 7: Future Forecast Visualization
# Filter to show last 60 days of actual + 30 days forecast
historical_tail = forecast_df[forecast_df['ds'] <= eval_df['ds'].max()].tail(60)
future_forecast = forecast_df[forecast_df['ds'] > eval_df['ds'].max()]

fig = go.Figure()

# Historical
fig.add_trace(go.Scatter(
    x=historical_tail['ds'],
    y=historical_tail['yhat'],
    mode='lines',
    name='Historical',
    line=dict(color='blue')
))

# Future forecast
fig.add_trace(go.Scatter(
    x=future_forecast['ds'],
    y=future_forecast['yhat'],
    mode='lines',
    name='Forecast (30 days)',
    line=dict(color='green', width=3)
))

# Confidence interval for forecast
fig.add_trace(go.Scatter(
    x=future_forecast['ds'],
    y=future_forecast['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=future_forecast['ds'],
    y=future_forecast['yhat_lower'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(0, 255, 0, 0.2)',
    fill='tonexty',
    name='Forecast Confidence'
))

fig.update_layout(
    title='Sales Forecast - Next 30 Days',
    xaxis_title='Date',
    yaxis_title='Predicted Sales ($)',
    height=500
)

fig.show()

predicted_30day = future_forecast['yhat'].sum()
print(f"\n 30-Day Revenue Forecast: ${predicted_30day:,.2f}")


 30-Day Revenue Forecast: $0.00


In [11]:
# Cell 8: Key Findings Summary
summary = f"""
=== MODEL EVALUATION SUMMARY ===

1. MODEL ACCURACY
   - MAPE: {mape:.2f}% ({quality})
   - RMSE: ${rmse:,.2f}
   - MAE: ${mae:,.2f}

2. PREDICTION QUALITY
   - {(abs(eval_df['residual_pct']) <= 10).mean() * 100:.1f}% of predictions within ±10%
   - {(abs(eval_df['residual_pct']) <= 20).mean() * 100:.1f}% of predictions within ±20%

3. 30-DAY FORECAST
   - Predicted revenue: ${predicted_30day:,.2f}
   - Daily average: ${future_forecast['yhat'].mean():,.2f}
   - Confidence range: ${future_forecast['yhat_lower'].sum():,.2f} - ${future_forecast['yhat_upper'].sum():,.2f}

4. MODEL CHARACTERISTICS
   - Captures weekly seasonality: Yes
   - Captures yearly trends: Yes
   - Handles outliers: Moderate

5. BUSINESS IMPLICATIONS
   - Can be used for inventory planning
   - Useful for revenue projections
   - Should be retrained monthly with new data

6. NEXT STEPS
   - Monitor actual vs predicted performance
   - A/B test against baseline models
   - Consider ensemble with XGBoost for better accuracy
"""

print(summary)

# Save summary
with open('../docs/model_performance.md', 'w') as f:
    f.write(summary)

print("\n Summary saved to docs/model_performance.md")


=== MODEL EVALUATION SUMMARY ===

1. MODEL ACCURACY
   - MAPE: 48.91% (Needs Improvement)
   - RMSE: $31,950.16
   - MAE: $21,457.46

2. PREDICTION QUALITY
   - 7.7% of predictions within ±10%
   - 38.5% of predictions within ±20%

3. 30-DAY FORECAST
   - Predicted revenue: $0.00
   - Daily average: $nan
   - Confidence range: $0.00 - $0.00

4. MODEL CHARACTERISTICS
   - Captures weekly seasonality: Yes
   - Captures yearly trends: Yes
   - Handles outliers: Moderate

5. BUSINESS IMPLICATIONS
   - Can be used for inventory planning
   - Useful for revenue projections
   - Should be retrained monthly with new data

6. NEXT STEPS
   - Monitor actual vs predicted performance
   - A/B test against baseline models
   - Consider ensemble with XGBoost for better accuracy


 Summary saved to docs/model_performance.md
